In [1]:
!nvidia-smi

Sat Dec 23 06:23:16 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# prompt: CPUのコア数をしらべて
import os
os.cpu_count()

2

In [3]:
class Config:
    name = "exp063" # 実験のたびにコピーしてここの名前を変えて実行するとoutputが別のファイルに保存される

    # Colab Env
    upload_from_colab = False
    api_path = "/content/drive/MyDrive/kaggle/kaggle.json"
    drive_path = "/content/drive/MyDrive/kaggle/ucbo"

DEBUG = False


In [4]:
import os
import json
import warnings
import shutil
import logging
import joblib
import random
import datetime
import sys


In [5]:
COLAB = "google.colab" in sys.modules
!pip install --upgrade --force-reinstall --no-deps kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.16-py3-none-any.whl size=110683 sha256=38a882bb71ef6660aee015ecad627d6fd7db0ead84b27ece8272c566d7f8c82d
  Stored in directory: /root/.cache/pip/wheels/43/4b/fb/736478af5e8004810081a06259f9aa2f7c3329fc5d03c2c412
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.16
    Uninstalling kaggle-1.5.16:
      Successfully uninstalled kaggle-1.5.16


In [6]:
if COLAB:
    print("This environment is Google Colab")

    # mount
    from google.colab import drive
    if not os.path.isdir("/content/drive"):
        drive.mount('/content/drive')


    # use kaggle api (need kaggle token)
    f = open(Config.api_path, 'r')
    json_data = json.load(f)
    os.environ["KAGGLE_USERNAME"] = json_data["username"]
    os.environ["KAGGLE_KEY"] = json_data["key"]

    # set dirs
    DRIVE = Config.drive_path
    EXP = Config.name
    INPUT = os.path.join(DRIVE, "Input")
    OUTPUT = os.path.join(DRIVE, "Output")
    SCRIPT = os.path.join(DRIVE, "Script")
    OUTPUT_EXP = os.path.join(OUTPUT, EXP)
    # EXP_MODEL = os.path.join(OUTPUT_EXP, "model")
    # EXP_FIG = os.path.join(OUTPUT_EXP, "fig")
    # EXP_PREDS = os.path.join(OUTPUT_EXP, "preds")

    # make dirs
    for d in [INPUT, SCRIPT, OUTPUT, OUTPUT_EXP]:
        os.makedirs(d, exist_ok=True)

    # if not os.path.isfile(os.path.join(INPUT, "train.csv")):
    #     # load dataset
    #     ! kaggle competitions download -c UBC-OCEAN -p $INPUT
    #     unzip_file = os.path.join(INPUT, 'UBC-OCEAN.zip')
    #     ! unzip $unzip_file -d $INPUT



This environment is Google Colab
Mounted at /content/drive


In [7]:
# !rm -rf /content/tmp

In [8]:
%%time
if not DEBUG:
    # tmp fileに直接ダウンロードする
    TMP = '/content/tmp'
    INPUT = '/content/tmp'
    !mkdir $TMP
    files = [
        'tyabanoamami/ucbo-colab-dataset', 'hmendonca/efficientnet-pytorch',
            #  'pjmathematician/ucbo-tiles-256-1', #'tyabanoamami/ucbo-clean-images',
            #  'pjmathematician/ucbo-tiles-256-2', 'pjmathematician/ucbo-tiles-256-3',
            #  'pjmathematician/ucbo-tiles-256-4', 'pjmathematician/ucbo-tiles-256-5',
        # 'jirkaborovec/tiles-of-cancer-2048px-scale-0-25',
        'tyabanoamami/ucbo-seg-tma',
    #    'tyabanoamami/ucbo-masktiles-dataset',
            ]
    for f in files:
        !kaggle datasets download -d $f -p $TMP
        t = f.split('/')[1]
        unzip_file = os.path.join(TMP, f'{t}.zip')
        unzip_file_ = TMP + '/' + t
        INPUT_ = INPUT + '/' + t
        ! unzip $unzip_file -d $unzip_file_
        # !mv $unzip_file_ $INPUT_
        !rm -rf $unzip_file

ストリーミング出力は最後の 5000 行に切り捨てられました。
  inflating: /content/tmp/ucbo-seg-tma/train_images/20316/00001_23-3.png  
  inflating: /content/tmp/ucbo-seg-tma/train_images/20316/00002_21-4.png  
  inflating: /content/tmp/ucbo-seg-tma/train_images/20316/00003_18-5.png  
  inflating: /content/tmp/ucbo-seg-tma/train_images/20316/00004_21-5.png  
  inflating: /content/tmp/ucbo-seg-tma/train_images/20316/00005_22-5.png  
  inflating: /content/tmp/ucbo-seg-tma/train_images/20316/00006_17-6.png  
  inflating: /content/tmp/ucbo-seg-tma/train_images/20316/00007_18-6.png  
  inflating: /content/tmp/ucbo-seg-tma/train_images/20316/00008_19-6.png  
  inflating: /content/tmp/ucbo-seg-tma/train_images/20316/00009_21-6.png  
  inflating: /content/tmp/ucbo-seg-tma/train_images/20316/00010_22-6.png  
  inflating: /content/tmp/ucbo-seg-tma/train_images/20316/00011_14-7.png  
  inflating: /content/tmp/ucbo-seg-tma/train_images/20316/00012_15-7.png  
  inflating: /content/tmp/ucbo-seg-tma/train_images/20316/00013_16-7

# Library

In [9]:
!pip install git+https://github.com/ildoonet/pytorch-gradual-warmup-lr.git
!pip install timm

  Cloning https://github.com/ildoonet/pytorch-gradual-warmup-lr.git to /tmp/pip-req-build-buzohcll
  Running command git clone --filter=blob:none --quiet https://github.com/ildoonet/pytorch-gradual-warmup-lr.git /tmp/pip-req-build-buzohcll
  Resolved https://github.com/ildoonet/pytorch-gradual-warmup-lr.git to commit 6b5e8953a80aef5b324104dc0c2e9b8c34d622bd
  Preparing metadata (setup.py) ... done
  Created wheel for warmup-scheduler: filename=warmup_scheduler-0.3.2-py3-none-any.whl size=3863 sha256=a5939e7bf1ef150c70a7d09f90e4aae7b961230f434072345943cd85b7d0197b
  Stored in directory: /tmp/pip-ephem-wheel-cache-5orywh0x/wheels/49/78/e6/9168d5844935482a171c7880a0626fa1c6c412b55666635f59
Successfully built warmup-scheduler
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.8 MB/s eta 0:00:00


In [10]:
import os
import sys
sys.path = [
    os.path.join(INPUT, 'efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master'),
] + sys.path

In [11]:
import time
import skimage.io
import numpy as np
import pandas as pd
import cv2
import PIL.Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler, SequentialSampler
from warmup_scheduler import GradualWarmupScheduler
from efficientnet_pytorch import model as enet
import albumentations
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.metrics import cohen_kappa_score
from tqdm.notebook import tqdm
from sklearn.metrics import balanced_accuracy_score
import timm


In [12]:
# df = pd.read_csv('/content/drive/MyDrive/kaggle/ucbo/Input/tiles-of-cancer-2048px-scale-0-25/train.csv')
# df[df.image_id==int(folders[1].split('/')[-1])].is_tma.values
# from glob import glob
# folders = glob('/content/drive/MyDrive/kaggle/ucbo/Input/tiles-of-cancer-2048px-scale-0-25/*')
# for f in sorted(folders, key=lambda x: int(x.split('/')[-1]) if x.split('/')[-1]!='train.csv' else 999999)[:-1]:
#     tma=False
#     if df[df.image_id==int(f.split('/')[-1])].is_tma.values[0]:
#         tma=True
#     print(f.split('/')[-1], len(glob(f+'/*png')), tma)

# Config

In [13]:
data_dir = os.path.join(INPUT, 'ucbo-seg-tma')
df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
image_folder = os.path.join(data_dir, 'train_images')
labels = ['CC', 'EC', 'HGSC', 'LGSC', 'MC', 'Other'] #df_train['label'].unique().tolist() + ['Other']
l2n = {v: k for k, v in enumerate(labels)}
n2l = {k: v for k, v in enumerate(labels)}

kernel_type = Config.name

enet_type = 'efficientnet_b0'
fold = 0
# tile_size = 512
image_size = 512
n_tiles = 4 if DEBUG else 4
batch_size = 16
num_workers = os.cpu_count()
out_dim = 5
init_lr = 1e-3
warmup_factor = 10

warmup_epo = 1
n_epochs = 1 if DEBUG else 20
df_train = df_train.sample(100).reset_index(drop=True) if DEBUG else df_train

device = torch.device('cuda')

print(image_folder)

import random
def seed_torch(seed=1029):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_torch()

/content/tmp/ucbo-seg-tma/train_images


# Create Folds

In [14]:
# exp060
import glob
seglist = glob.glob(os.path.join(INPUT, 'ucbo-seg-tma/train_images/*'))
seglist = [int(i.split('/')[-1]) for i in seglist]
df_train = df_train[df_train['image_id'].isin(seglist)].reset_index(drop=True)
# df_train = df_train.query('tma_train==True')

In [15]:
skf = StratifiedKFold(5, shuffle=True, random_state=42)
df_train['fold'] = -1
# df_train.loc[df_train['is_tma']==True, 'fold'] = 0
for i, (train_idx, valid_idx) in enumerate(skf.split(df_train, df_train['label'])):
    df_train.loc[valid_idx, 'fold'] = i
df_train.head()

,image_id,label,image_width,image_height,is_tma,fold
0,66,LGSC,48871,48195,False,0
1,91,HGSC,3388,3388,True,0
2,1020,HGSC,36585,33751,False,1
3,1101,HGSC,26306,18403,False,1
4,1252,HGSC,60420,27480,False,0


In [16]:
def get_image_path(image_id:int):
    path = os.path.join(INPUT, 'ucbo-seg-tma/train_images')
    return os.path.join(path, str(image_id))

df_train['tile_path'] = df_train['image_id'].apply(lambda x: get_image_path(x))

df_train['total_tiles'] = df_train['tile_path'].apply(lambda x: len(os.listdir(x)))
df_train.head()

# train['clean_tiles'] = train['image_id'].apply(lambda x: len(d[x]))/train['total_tiles']
# train.head()

,image_id,label,image_width,image_height,is_tma,fold,tile_path,total_tiles
0,66,LGSC,48871,48195,False,0,/content/tmp/ucbo-seg-tma/train_images/66,88
1,91,HGSC,3388,3388,True,0,/content/tmp/ucbo-seg-tma/train_images/91,4
2,1020,HGSC,36585,33751,False,1,/content/tmp/ucbo-seg-tma/train_images/1020,30
3,1101,HGSC,26306,18403,False,1,/content/tmp/ucbo-seg-tma/train_images/1101,5
4,1252,HGSC,60420,27480,False,0,/content/tmp/ucbo-seg-tma/train_images/1252,5


In [17]:
df_train.total_tiles.describe()

count    177.000000
mean      37.378531
std       40.200466
min        2.000000
25%        6.000000
50%       20.000000
75%       61.000000
max      198.000000
Name: total_tiles, dtype: float64

# Model

In [18]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# class MishFunction(torch.autograd.Function):
#     @ staticmethod
#     def forward(ctx, x):
#         ctx.save_for_backward(x)
#         return x * torch.tanh(F.softplus(x))   # x * tanh(ln(1 + exp(x)))

#     @ staticmethod １ますあいてる
#     def backward(ctx, grad_output):
#         x = ctx.saved_variables[0]
#         sigmoid = torch.sigmoid(x)
#         tanh_sp = torch.tanh(F.softplus(x))
#         return grad_output * (tanh_sp + x * sigmoid * (1 - tanh_sp * tanh_sp))

# class Mish(nn.Module):
#     def forward(self, x):
#         return MishFunction.apply(x)

# def to_Mish(model):
#     for child_name, child in model.named_children():
#         if isinstance(child, nn.ReLU):
#             setattr(model, child_name, Mish())
#         else:
#             to_Mish(child)


class AdaptiveConcatPool2d(nn.Module):
    def __init__(self):
        super(AdaptiveConcatPool2d, self).__init__()
        self.avg = nn.AdaptiveAvgPool2d(1)
        self.max = nn.AdaptiveMaxPool2d(1)
    def forward(self, x):
        x1 = self.avg(x)
        x2 = self.max(x)
        x = torch.cat([x1, x2], 1)
        return x


class Model(nn.Module):
    def __init__(self, arch='efficientnet_b0', n=out_dim, pre=True):
        super().__init__()
        m = timm.create_model(arch, pretrained=pre)
        self.enc = nn.Sequential(*list(m.children())[:-2])
        nc = list(m.children())[-1].in_features
        # self.AdaptiveConcatPool2d = AdaptiveConcatPool2d()
        self.head = nn.Sequential(nn.AdaptiveAvgPool2d(1),nn.Flatten(),nn.Linear(nc,512),
                            nn.BatchNorm1d(512), nn.SiLU(), nn.Linear(512,n))


    def forward(self, x):
        shape = x.shape
        n = shape[1]
        x = x.view(-1,shape[2],shape[3],shape[4])
        x = self.enc(x)

        shape = x.shape
        x = x.view(-1,n,shape[1],shape[2],shape[3]).permute(0,2,1,3,4).contiguous()\
          .view(-1,shape[1],shape[2]*n,shape[3])
        x = self.head(x)
        return x

In [19]:
# model = enetv2('efficientnet_b0', 6)
# model = timm.create_model('efficientnet_b0', pretrained=True)
# model.__dict__['default_cfg']

# Dataset

In [20]:
import joblib
good_images = joblib.load(os.path.join(OUTPUT, 'bw_checker/good_images.pkl'))

In [21]:
if DEBUG:
    for k, v in good_images.items():
        tiles = [t.replace('/content/tmp', INPUT) for t in v]
        good_images[k] = tiles
        if len(tiles) < 13:
            print(k, len(tiles))

In [22]:
def get_tiles(paths, img_sz):
    ret = []
    for path in paths:
        # PNGファイルを読み込み
        image = PIL.Image.open(path).resize((img_sz, img_sz))

        # Pillow ImageオブジェクトをNumPy配列に変換
        image_array = np.array(image)
        ret.append(image_array)
    return ret


class PANDADataset(Dataset):
    def __init__(self,
                 df,
                 image_size,
                 n_tiles=n_tiles,
                 tile_mode=0,
                 rand=False,
                 transform=None,
                ):

        self.df = df.reset_index(drop=True)
        self.image_size = image_size
        self.n_tiles = n_tiles
        self.tile_mode = tile_mode
        self.rand = rand
        self.transform = transform

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        row = self.df.iloc[index]
        img_id = row.image_id


        # tiles = good_images[img_id]

        files = os.listdir(row.tile_path)
        tiles = [row.tile_path + f'/{e}' for e in files]

        n_imgs = len(tiles)
        tiles = get_tiles(tiles, self.image_size)

        if self.rand:
            idxes = np.random.choice(list(range(self.n_tiles)), self.n_tiles, replace=False)
        else:
            idxes = list(range(self.n_tiles))

        n_row_tiles = int(np.sqrt(self.n_tiles))

        images = []
        for i in range(self.n_tiles):

            try:
                this_img = tiles[idxes[i%n_imgs]]
            except:
                this_img = np.ones((self.image_size, self.image_size, 3)).astype(np.uint8) * 255

            # this_img = 255 - this_img
            if self.transform is not None:
                this_img = self.transform(image=this_img)['image']

            this_img = this_img.astype(np.float32)
            this_img = this_img.transpose(2, 0, 1)
            images.append(this_img)


        label = np.zeros(out_dim).astype(np.float32)
        label[l2n[row.label]] = 1.
        return torch.tensor(np.array(images)), torch.tensor(label)



# Augmentations

In [23]:
transforms_train = albumentations.Compose([
    albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], # 上のimages /= 255に注意
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0,
                p=1.0),
    albumentations.Transpose(p=0.5),
    albumentations.VerticalFlip(p=0.5),
    albumentations.HorizontalFlip(p=0.5),
    albumentations.RandomBrightnessContrast(p=0.75),
    albumentations.ShiftScaleRotate(p=0.75),
    albumentations.OneOf([
    # albumentations.GaussNoise(var_limit=[10, 50]),
    albumentations.GaussianBlur(),
    albumentations.MotionBlur(),
    ], p=0.3),
    albumentations.GridDistortion(num_steps=5, distort_limit=0.3, p=0.5),
    albumentations.CoarseDropout(max_holes=1, max_width=int(512* 0.2), max_height=int(512* 0.2),
    mask_fill_value=0, p=0.5)
])
transforms_val = albumentations.Compose([
    albumentations.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0,
                p=1.0),
    ])

# efficientnetb0
# 'mean': (0.485, 0.456, 0.406),
#  'std': (0.229, 0.224, 0.225),

In [24]:
dataset_show = PANDADataset(df_train.query('is_tma==True'), image_size, n_tiles, 0, transform=transforms_val)
from pylab import rcParams
rcParams['figure.figsize'] = 20,10
for i in range(5):
    f, axarr = plt.subplots(1,4)
    idx = np.random.randint(0, len(dataset_show))
    for p in range(4):
        img, label = dataset_show[idx]
        axarr[p].imshow(img[p].transpose(0, 1).transpose(1,2).squeeze())
        axarr[p].set_title(label)


Output hidden; open in https://colab.research.google.com to view.

# Loss

In [25]:
# labels = ['CC', 'EC', 'HGSC', 'LGSC', 'MC', 'Other'] #df_train['label'].unique().tolist() + ['Other']
# pos_weight = torch.tensor([1.0, 3.0, 3.0, 3.0, 1.0], dtype=torch.float32)
# pos_weight = pos_weight.to(device)
# criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

criterion = nn.BCEWithLogitsLoss()
# criterion = nn.CrossEntropyLoss()

# Train and Val

In [26]:
# def train_epoch(loader, optimizer):

#     model.train()
#     train_loss = []
#     bar = tqdm(loader)
#     for (data, target) in bar:

#         data, target = data.to(device), target.to(device)
#         loss_func = criterion
#         optimizer.zero_grad()
#         logits = model(data)
#         loss = loss_func(logits, target)
#         loss.backward()
#         optimizer.step()

#         loss_np = loss.detach().cpu().numpy()
#         train_loss.append(loss_np)
#         smooth_loss = sum(train_loss[-100:]) / min(len(train_loss), 100)
#         bar.set_description('loss: %.5f, smth: %.5f' % (loss_np, smooth_loss))
#     return train_loss


# prompt: def train_epoch(loader, optimizer):      model.train()     train_loss = []     bar = tqdm(loader)     for (data, target) in bar:          data, target = data.to(device), target.to(device)         loss_func = criterion         optimizer.zero_grad()         logits = model(data)         loss = loss_func(logits, target)         loss.backward()         optimizer.step()          loss_np = loss.detach().cpu().numpy()         train_loss.append(loss_np)         smooth_loss = sum(train_loss[-100:]) / min(len(train_loss), 100)         bar.set_description('loss: %.5f, smth: %.5f' % (loss_np, smooth_loss))     return train_lossをtorch.autocast and torch.cuda.amp.GradScalerをつかって書き換えて

def train_epoch(loader, optimizer):
    model.train()
    train_loss = []
    bar = tqdm(loader)
    scaler = torch.cuda.amp.GradScaler()
    for (data, target) in bar:
        data, target = data.to(device), target.to(device)
        loss_func = criterion
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            logits = model(data)
            loss = loss_func(logits, target)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        loss_np = loss.detach().cpu().numpy()
        train_loss.append(loss_np)
        smooth_loss = sum(train_loss[-100:]) / min(len(train_loss), 100)
        bar.set_description('loss: %.5f, smth: %.5f' % (loss_np, smooth_loss))
    return train_loss


def val_epoch(loader, get_output=False):

    model.eval()
    val_loss = []
    LOGITS = []
    PREDS = []
    TARGETS = []

    with torch.no_grad():
        for (data, target) in tqdm(loader):
            data, target = data.to(device), target.to(device)
            logits = model(data)

            loss = criterion(logits, target)

            pred = logits.sigmoid().detach()
            LOGITS.append(logits)
            PREDS.append(pred)
            TARGETS.append(target)

            val_loss.append(loss.detach().cpu().numpy())
        val_loss = np.mean(val_loss)

    LOGITS = torch.cat(LOGITS).cpu().numpy()
    PREDS = torch.cat(PREDS).cpu().numpy()
    TARGETS = torch.cat(TARGETS).cpu().numpy()
#     acc = (PREDS == TARGETS).mean() * 100.
    comp_metric = balanced_accuracy_score(TARGETS.argmax(1), PREDS.argmax(1))
    print(comp_metric)

#     qwk = cohen_kappa_score(PREDS, TARGETS, weights='quadratic')
#     qwk_k = cohen_kappa_score(PREDS[df_valid['data_provider'] == 'karolinska'], df_valid[df_valid['data_provider'] == 'karolinska'].isup_grade.values, weights='quadratic')
#     qwk_r = cohen_kappa_score(PREDS[df_valid['data_provider'] == 'radboud'], df_valid[df_valid['data_provider'] == 'radboud'].isup_grade.values, weights='quadratic')
#     print('qwk', qwk, 'qwk_k', qwk_k, 'qwk_r', qwk_r)

    if get_output:
        return LOGITS
    else:
        return val_loss, comp_metric, PREDS



# Create Dataloader & Model & Optimizer

In [27]:
train_idx = np.where((df_train['fold'] != fold))[0]
valid_idx = np.where((df_train['fold'] == fold))[0]

df_this  = df_train.loc[train_idx]
df_valid = df_train.loc[valid_idx]

dataset_train = PANDADataset(df_this , image_size, n_tiles, transform=transforms_train)
dataset_valid = PANDADataset(df_valid, image_size, n_tiles, transform=transforms_val)

train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, sampler=RandomSampler(dataset_train), num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(dataset_valid, batch_size=batch_size, sampler=SequentialSampler(dataset_valid), num_workers=num_workers)

model = Model(arch='efficientnet_b0', n=5, pre=True)
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=init_lr/warmup_factor)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs-warmup_epo, eta_min=1.0e-4)
scheduler = GradualWarmupScheduler(optimizer, multiplier=warmup_factor, total_epoch=warmup_epo, after_scheduler=scheduler_cosine)

print(len(dataset_train), len(dataset_valid))

model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

141 36


In [28]:
dataset_train[0][0].shape

torch.Size([4, 3, 512, 512])

# Run Training

In [29]:
last_layer = list(model.children())[-1]
print(f'except last layer: {last_layer}')
for param in model.parameters():
    param.requires_grad = False

for param in last_layer.parameters():
    param.requires_grad = True

except last layer: Sequential(
  (0): AdaptiveAvgPool2d(output_size=1)
  (1): Flatten(start_dim=1, end_dim=-1)
  (2): Linear(in_features=1280, out_features=512, bias=True)
  (3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): SiLU()
  (5): Linear(in_features=512, out_features=5, bias=True)
)


In [30]:
def mymetric(df):
    cols = df['label'].unique().tolist()
    recalls = dict()
    for col in cols:
        df_ = df.query('label==@col')
        recalls[col] = (df_['label']==df_['pred']).sum()/df_.shape[0]
    score = 0
    for k, v in recalls.items():
        score += v
    score /= len(cols)
    return score, recalls

In [31]:
%%time
comp_metric_max = 0.
best_file = os.path.join(OUTPUT_EXP, f'{kernel_type}_best_fold{fold}.pth')
for epoch in range(1, n_epochs+1):
    print(time.ctime(), 'Epoch:', epoch)
    scheduler.step(epoch-1)

    train_loss = train_epoch(train_loader, optimizer)
    val_loss, comp_metric, oof_preds = val_epoch(valid_loader)
    df_valid['pred'] = oof_preds.argmax(1)
    df_valid['pred'] = df_valid['pred'].map(n2l)

    content = time.ctime() + ' ' + f'Epoch {epoch}, lr: {optimizer.param_groups[0]["lr"]:.7f}, train loss: {np.mean(train_loss):.5f}, val loss: {np.mean(val_loss):.5f}, balanced accuracy: {(comp_metric):.5f}, '
    content += str(mymetric(df_valid))
    print(content)
    with open(os.path.join(OUTPUT_EXP, f'log_{kernel_type}.txt'), 'a') as appender:
        appender.write(content + '\n')

    if comp_metric > comp_metric_max:
        print('score2 ({:.6f} --> {:.6f}).  Saving model ...'.format(comp_metric_max, comp_metric))
        torch.save(model.state_dict(), best_file)
        np.save(os.path.join(OUTPUT_EXP, f'oof_preds_best_fold{fold}.npy'), oof_preds)
        comp_metric_max = comp_metric

torch.save(model.state_dict(), os.path.join(OUTPUT_EXP, f'{kernel_type}_final_fold{fold}.pth'))
np.save(os.path.join(OUTPUT_EXP, f'oof_preds_final_fold{fold}.npy'), oof_preds)
print(f'comp metric max {comp_metric_max}')

Sat Dec 23 06:28:28 2023 Epoch: 1


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://githu

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

0.20192307692307693
Sat Dec 23 06:30:02 2023 Epoch 1, lr: 0.0001000, train loss: 0.66977, val loss: 0.69173, balanced accuracy: 0.20192, (0.20192307692307693, {'LGSC': 0.5, 'HGSC': 0.38461538461538464, 'MC': 0.0, 'EC': 0.125, 'CC': 0.0})
score2 (0.000000 --> 0.201923).  Saving model ...
Sat Dec 23 06:30:05 2023 Epoch: 2


...

0.41593406593406596
Sat Dec 23 06:54:10 2023 Epoch 19, lr: 0.0001244, train loss: 0.13031, val loss: 0.36558, balanced accuracy: 0.41593, (0.41593406593406596, {'LGSC': 0.0, 'HGSC': 0.6153846153846154, 'MC': 0.5, 'EC': 0.25, 'CC': 0.7142857142857143})
Sat Dec 23 06:54:10 2023 Epoch: 20


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

0.46593406593406594
Sat Dec 23 06:55:30 2023 Epoch 20, lr: 0.0001061, train loss: 0.13515, val loss: 0.36218, balanced accuracy: 0.46593, (0.46593406593406594, {'LGSC': 0.25, 'HGSC': 0.6153846153846154, 'MC': 0.5, 'EC': 0.25, 'CC': 0.7142857142857143})
comp metric max 0.5873626373626374
CPU times: user 1min 3s, sys: 22.4 s, total: 1min 25s
Wall time: 27min 3s


In [32]:
# 255-val 0.5752747252747252

In [33]:
!nvidia-smi

Sat Dec 23 06:55:32 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   79C    P0              46W /  70W |   6361MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [34]:
#gpu 14GB